In [1]:
import polars as pl
from pathlib import Path
from backend.config import Settings, DFType
from backend.db_tools import DBEngine
from backend.crud import select_table_from_db, fetch_data
from backend import models

from enum import Enum
from sqlalchemy import select


# Create connection
db_engine = DBEngine(**Settings().model_dump())


## Get all totals per region, per year

In [2]:
stmt = (
    select(models.Regios.regio, models.Regios.regio_key, models.Perioden.jaar, models.Bevolking.bevolking_1_januari, models.Geslacht.geslacht, models.CategoryGroup.catgroup, models.Burgstaat.burgerlijkestaat)
    .join(models.Geslacht, models.Bevolking.geslacht_key == models.Geslacht.geslacht_key)
    .join(models.Perioden, models.Bevolking.datum_key == models.Perioden.datum_key)
    .join(models.Regios, models.Bevolking.regio_key == models.Regios.regio_key)
    .join(models.Leeftijd, models.Bevolking.leeftijd_key == models.Leeftijd.leeftijd_key)
    .join(models.CategoryGroup, models.Leeftijd.categorygroupid == models.CategoryGroup.catgroup_key)
    .join(models.Burgstaat, models.Bevolking.burgst_key == models.Burgstaat.burgst_key)
    .join(models.Bodemgebruik, (models.Bevolking.regio_key == models.Bodemgebruik.regio_key) & (models.Bevolking.datum_key == models.Bodemgebruik.datum_key))
    # .where(models.Regios.regio == "Nederland")
    .where(models.CategoryGroup.catgroup == "Totaal")
    .where(models.Burgstaat.burgerlijkestaat == "Totaal burgerlijke staat")
    .where(models.Geslacht.geslacht == "Totaal mannen en vrouwen")
)
df = fetch_data(stmt=stmt, db_engine=db_engine, package=DFType.POLARS)

SELECT regios.regio, regios.regio_key, perioden.jaar, bevolking.bevolking_1_januari, geslacht.geslacht, categorygroup.catgroup, burgstaat.burgerlijkestaat 
FROM bevolking JOIN geslacht ON bevolking.geslacht_key = geslacht.geslacht_key JOIN perioden ON bevolking.datum_key = perioden.datum_key JOIN regios ON bevolking.regio_key = regios.regio_key JOIN leeftijd ON bevolking.leeftijd_key = leeftijd.leeftijd_key JOIN categorygroup ON leeftijd.categorygroupid = categorygroup.catgroup_key JOIN burgstaat ON bevolking.burgst_key = burgstaat.burgst_key JOIN bodemgebruik ON bevolking.regio_key = bodemgebruik.regio_key AND bevolking.datum_key = bodemgebruik.datum_key 
WHERE categorygroup.catgroup = 'Totaal' AND burgstaat.burgerlijkestaat = 'Totaal burgerlijke staat' AND geslacht.geslacht = 'Totaal mannen en vrouwen'


### Create a top 5 list of gemeentes with the highest population

In [3]:
devdf = df.clone()

devdf = devdf.filter(pl.col('jaar') == pl.col('jaar').max())
devdf = devdf.filter(devdf['regio_key'].str.starts_with('GM'))

In [4]:
sorted_df = devdf.sort('bevolking_1_januari', descending=True)
top_5_df = sorted_df.head(5)
top_5_df

regio,regio_key,jaar,bevolking_1_januari,geslacht,catgroup,burgerlijkestaat
str,str,i64,i64,str,str,str
"""Amsterdam""","""GM0363""",2017,844947,"""Totaal mannen …","""Totaal""","""Totaal burgerl…"
"""Rotterdam""","""GM0599""",2017,634660,"""Totaal mannen …","""Totaal""","""Totaal burgerl…"
"""'s-Gravenhage …","""GM0518""",2017,524882,"""Totaal mannen …","""Totaal""","""Totaal burgerl…"
"""Utrecht (gemee…","""GM0344""",2017,343038,"""Totaal mannen …","""Totaal""","""Totaal burgerl…"
"""Eindhoven""","""GM0772""",2017,226868,"""Totaal mannen …","""Totaal""","""Totaal burgerl…"


### Get top 5 gemeentes with the highest growth since the previous measuring moment

Some gemeentes have been discontinued. So, we will only look at the gemeentes that are still active in the most recent year.

In [5]:
devdf = df.clone()
devdf = devdf.filter(devdf['regio_key'].str.starts_with('GM'))
active_gemeentes = devdf.filter(pl.col('jaar') == pl.col('jaar').max())
active_gemeentes = active_gemeentes.drop_nulls('bevolking_1_januari')[['regio', 'regio_key']]
devdf = devdf.filter(devdf['regio'].is_in(active_gemeentes['regio']))

Now we calculate the growth per gemeente.

In [6]:
devdf = devdf.with_columns((pl.col('bevolking_1_januari').shift(1)).over('regio').alias('previous_moment'))
devdf = devdf.with_columns(((pl.col('bevolking_1_januari') - pl.col('previous_moment'))/pl.col('previous_moment')*100).alias('percentage_growth'))
devdf = devdf.with_columns((pl.col('bevolking_1_januari') - pl.col('previous_moment')).alias('absolute_growth'))

Calculate the top 5 for both the highest relative growth and the highest absolute growth.

In [7]:
devdf_2023 = devdf.filter(pl.col('jaar') == pl.col('jaar').max())
sorted_df = devdf_2023.sort('percentage_growth', descending=True)
top_5_relative_growth = sorted_df.head(5)
sorted_df = devdf_2023.sort('absolute_growth', descending=True)
top_5_absolute_growth = sorted_df.head(5)

In [8]:
top_5_relative_growth

regio,regio_key,jaar,bevolking_1_januari,geslacht,catgroup,burgerlijkestaat,previous_moment,percentage_growth,absolute_growth
str,str,i64,i64,str,str,str,i64,f64,i64
"""Edam-Volendam""","""GM0385""",2017,35800,"""Totaal mannen …","""Totaal""","""Totaal burgerl…",29087,23.079039,6713
"""Blaricum""","""GM0376""",2017,10201,"""Totaal mannen …","""Totaal""","""Totaal burgerl…",9312,9.546821,889
"""Noordenveld""","""GM1699""",2017,32981,"""Totaal mannen …","""Totaal""","""Totaal burgerl…",31137,5.922215,1844
"""Rijswijk (ZH.)…","""GM0603""",2017,51027,"""Totaal mannen …","""Totaal""","""Totaal burgerl…",48216,5.830015,2811
"""Oostzaan""","""GM0431""",2017,9652,"""Totaal mannen …","""Totaal""","""Totaal burgerl…",9187,5.0615,465


In [9]:
top_5_absolute_growth

regio,regio_key,jaar,bevolking_1_januari,geslacht,catgroup,burgerlijkestaat,previous_moment,percentage_growth,absolute_growth
str,str,i64,i64,str,str,str,i64,f64,i64
"""Amsterdam""","""GM0363""",2017,844947,"""Totaal mannen …","""Totaal""","""Totaal burgerl…",821752,2.822628,23195
"""Rotterdam""","""GM0599""",2017,634660,"""Totaal mannen …","""Totaal""","""Totaal burgerl…",623652,1.765087,11008
"""'s-Gravenhage …","""GM0518""",2017,524882,"""Totaal mannen …","""Totaal""","""Totaal burgerl…",514861,1.946351,10021
"""Utrecht (gemee…","""GM0344""",2017,343038,"""Totaal mannen …","""Totaal""","""Totaal burgerl…",334176,2.651896,8862
"""Edam-Volendam""","""GM0385""",2017,35800,"""Totaal mannen …","""Totaal""","""Totaal burgerl…",29087,23.079039,6713


## Find out if this growth can be explained by bodemgebruik
Get the bodemgebruik data for the top 5 gemeentes from the database, and merge with the dataframes.

In [10]:
top_regio = top_5_absolute_growth['regio'].to_list()
top_regio.extend(top_5_relative_growth['regio'].to_list())

In [11]:
stmt = (
    select(models.Regios.regio, models.Regios.regio_key, models.Perioden.jaar, models.Bodemgebruik)
    .join(models.Perioden, models.Bodemgebruik.datum_key == models.Perioden.datum_key)
    .join(models.Regios, models.Bodemgebruik.regio_key == models.Regios.regio_key)
    .filter(models.Regios.regio.in_(top_regio))
)
bodemgebruik = fetch_data(stmt=stmt, db_engine=db_engine, package=DFType.POLARS)

SELECT regios.regio, regios.regio_key, perioden.jaar, bodemgebruik.id, bodemgebruik.regio_key AS regio_key_1, bodemgebruik.datum_key, bodemgebruik.totale_oppervlakte, bodemgebruik.totaal_verkeersterrein, bodemgebruik.spoorterrein, bodemgebruik.wegverkeersterrein, bodemgebruik.vliegveld, bodemgebruik.totaal_bebouwd_terrein, bodemgebruik.woonterrein, bodemgebruik.terrein_voor_detailhandel_en_horeca, bodemgebruik.terrein_voor_openbare_voorzieningen, bodemgebruik.terrein_voor_sociaal_culturele_voorz, bodemgebruik.bedrijventerrein, bodemgebruik.totaal_semi_bebouwd_terrein, bodemgebruik.stortplaats, bodemgebruik.wrakkenopslagplaats, bodemgebruik.begraafplaats, bodemgebruik.delfstofwinplaats, bodemgebruik.bouwterrein, bodemgebruik.semi_verhard_overig_terrein, bodemgebruik.totaal_recreatieterrein, bodemgebruik.park_en_plantsoen, bodemgebruik.sportterrein, bodemgebruik.volkstuin, bodemgebruik.dagrecreatief_terrein, bodemgebruik.verblijfsrecreatief_terrein, bodemgebruik.totaal_agrarisch_terrein,

In [12]:
bodemgebruik_df = bodemgebruik.clone()
bodemgebruik_df.filter(pl.col('jaar') == pl.col('jaar').max())

regio,regio_key,jaar,id,regio_key_1,datum_key,totale_oppervlakte,totaal_verkeersterrein,spoorterrein,wegverkeersterrein,vliegveld,totaal_bebouwd_terrein,woonterrein,terrein_voor_detailhandel_en_horeca,terrein_voor_openbare_voorzieningen,terrein_voor_sociaal_culturele_voorz,bedrijventerrein,totaal_semi_bebouwd_terrein,stortplaats,wrakkenopslagplaats,begraafplaats,delfstofwinplaats,bouwterrein,semi_verhard_overig_terrein,totaal_recreatieterrein,park_en_plantsoen,sportterrein,volkstuin,dagrecreatief_terrein,verblijfsrecreatief_terrein,totaal_agrarisch_terrein,terrein_voor_glastuinbouw,overig_agrarisch_terrein,totaal_bos_en_open_natuurlijk_terrein,bos,open_droog_natuurlijk_terrein,open_nat_natuurlijk_terrein,totaal_binnenwater,ijsselmeer_markermeer,afgesloten_zeearm,rijn_en_maas,randmeer,spaarbekken,recreatief_binnenwater,binnenwater_voor_delfstofwinning,vloei_en_of_slibveld,overig_binnenwater,totaal_buitenwater,waddenzee_eems_dollard,oosterschelde,westerschelde,noordzee
str,str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,bool,i64,i64,i64,i64,i64,i64,i64,i64,bool,bool,bool,i64
"""Blaricum""","""GM0376""",2017,1628,"""GM0376""","""2017JJ00""",1556,56,0,56,0,352,329,7,5,9,2,46,0,0,2,0,44,0,58,30,15,0,11,2,332,0,332,264,146,94,23,449,null,null,null,444,0,0,0,0,5,null,null,null,null,null
"""Amsterdam""","""GM0363""",2017,1160,"""GM0363""","""2017JJ00""",21949,1424,325,1099,0,8244,4627,510,159,424,2524,937,0,13,109,0,773,42,2701,1520,696,323,145,18,2737,28,2709,496,241,16,239,5409,2288,null,null,null,0,371,0,1,2748,null,null,null,null,null
"""Edam-Volendam""","""GM0385""",2017,2366,"""GM0385""","""2017JJ00""",8000,202,25,177,0,576,467,16,5,3,85,59,4,0,4,0,36,16,153,58,48,10,12,24,4391,0,4391,51,0,0,51,2567,2266,null,null,null,0,15,0,0,287,null,null,null,null,null
"""'s-Gravenhage …","""GM0518""",2017,2870,"""GM0518""","""2017JJ00""",9813,621,84,538,0,4843,3517,294,171,256,604,190,0,0,58,0,122,11,1326,662,430,101,88,45,177,9,168,1089,424,665,0,316,null,null,null,null,0,81,0,0,234,1252,null,null,null,1252
"""Noordenveld""","""GM1699""",2017,4742,"""GM1699""","""2017JJ00""",20529,336,0,336,0,1014,720,28,103,32,131,103,0,3,14,59,27,0,400,31,130,5,23,211,13000,0,13000,5049,2591,245,2213,628,null,null,null,null,0,12,35,0,581,null,null,null,null,null
"""Oostzaan""","""GM0431""",2017,5003,"""GM0431""","""2017JJ00""",1608,60,12,48,0,188,146,4,9,0,28,5,0,0,1,0,3,0,177,6,11,0,160,0,512,0,512,214,60,0,154,452,null,null,null,null,0,116,0,0,336,null,null,null,null,null
"""Rotterdam""","""GM0599""",2017,5489,"""GM0599""","""2017JJ00""",32416,2184,571,1559,54,12081,4689,406,299,353,6334,1959,74,19,107,10,1488,260,2536,1584,492,249,169,41,2320,150,2169,765,157,497,111,7561,null,null,1908,null,4,226,0,157,5266,3011,null,null,null,3011
"""Rijswijk (ZH.)…","""GM0603""",2017,5399,"""GM0603""","""2017JJ00""",1449,112,9,103,0,710,450,34,28,39,159,68,0,0,14,0,54,0,422,253,120,32,17,0,81,8,73,4,4,0,0,53,null,null,null,null,0,29,0,0,24,null,null,null,null,null
"""Utrecht (gemee…","""GM0344""",2017,6209,"""GM0344""","""2017JJ00""",9921,951,156,796,0,4247,2592,218,140,331,966,296,0,0,50,0,246,0,1277,681,400,71,120,4,2417,8,2409,189,169,16,4,544,null,null,null,null,0,180,0,0,364,null,null,null,null,null


In [32]:
ls = bodemgebruik_df['regio'].unique().to_list()
bodemgebruik_df.filter(bodemgebruik_df['regio'].is_in(ls[:2]))['regio_key'].unique().to_list()

['GM0385', 'GM0431']

In [33]:
stmt = (
    select(models.BevolkingDescribed)
)
df = fetch_data(stmt=stmt, db_engine=db_engine, package=DFType.POLARS)

AttributeError: module 'backend.models' has no attribute 'BevolkingDescribed'